# Keras Callbacks
- Keras Callbacks provide useful tools to babysit training process
    - ModelCheckpoint
    - Earlystopping
    - ReduceLROnPlateau

In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *

### Load Dataset

In [3]:
data = load_digits()

In [4]:
X_data = data.images
y_data = data.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3, random_state = 777)

In [6]:
# reshaping X data => flatten into 1-dimensional
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [7]:
# converting y data into categorical (one-hot encoding)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1257, 64)
(540, 64)
(1257, 10)
(540, 10)


## 1. ModelCheckpoint
- **ModelCheckpoint** is used to 'checkpoint' model results on training
- Oftentimes, it is used to save only best model

In [9]:
def create_model():
    model = Sequential()
    model.add(Dense(100, input_shape = (X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dense(y_train.shape[1]))
    model.add(Activation('sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [10]:
model = create_model()

### Creating callbacks list
- ModelCheckpoint instances are stored in list and passed on when training 

In [11]:
callbacks = [ModelCheckpoint(filepath = 'saved_model.hdf5', monitor='val_acc', verbose=1, mode='max')]

In [12]:
model.fit(X_train, y_train, epochs = 10, batch_size = 500, callbacks = callbacks, validation_data = (X_test, y_test))

Epoch 1/10
1/3 [=========>....................] - ETA: 0s - loss: 3.0892 - accuracy: 0.1000
Epoch 00001: saving model to saved_model.hdf5
3/3 [==============================] - 0s 72ms/step - loss: 2.8432 - accuracy: 0.0899 - val_loss: 2.2683 - val_accuracy: 0.0796
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 2.2550 - accuracy: 0.0740
Epoch 00002: saving model to saved_model.hdf5
3/3 [==============================] - 0s 18ms/step - loss: 2.2042 - accuracy: 0.0835 - val_loss: 2.1644 - val_accuracy: 0.1037
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 2.1482 - accuracy: 0.1100
Epoch 00003: saving model to saved_model.hdf5
3/3 [==============================] - 0s 16ms/step - loss: 2.1342 - accuracy: 0.1257 - val_loss: 2.1116 - val_accuracy: 0.1611
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 2.0909 - accuracy: 0.2000
Epoch 00004: saving model to saved_model.hdf5
3/3 [==============================] - 0s 19ms/step - loss: 2.07

In [13]:
results = model.evaluate(X_test, y_test)

17/17 [==============================] - 0s 1ms/step - loss: 1.1874 - accuracy: 0.3815


In [14]:
print('Accuracy: ', results[1])

Accuracy:  0.38148146867752075


### Loading saved weights
- Saved weights can be loaded and used without further training
- This is especially useful when training time is long and model has to be reused a number of times

In [15]:
another_model = create_model()
another_model.load_weights('saved_model.hdf5')
another_model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
results = another_model.evaluate(X_test, y_test)

17/17 [==============================] - 0s 1ms/step - loss: 1.1874 - accuracy: 0.3815


In [17]:
print('Accuracy: ', results[1])

Accuracy:  0.38148146867752075


### Selecting best model
- Best model during whole epoch can be selected using ModelCheckpoint
    - Set **'save_best_only'** parameter as True
- Usually, validation accuracy (val acc) is monitored and used as criterion for best model

In [32]:
callbacks = [ModelCheckpoint(filepath = 'best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only = True, mode='max')]

In [36]:
model = create_model()

In [37]:
model.fit(X_train, y_train, epochs = 10, batch_size = 500, callbacks = callbacks, validation_data = (X_test, y_test))

Epoch 1/10
1/3 [=========>....................] - ETA: 0s - loss: 3.7057 - accuracy: 0.1000
Epoch 00001: val_accuracy did not improve from 0.97407
3/3 [==============================] - 0s 45ms/step - loss: 3.1909 - accuracy: 0.1058 - val_loss: 2.4031 - val_accuracy: 0.0944
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 2.3462 - accuracy: 0.1080
Epoch 00002: val_accuracy did not improve from 0.97407
3/3 [==============================] - 0s 10ms/step - loss: 2.2394 - accuracy: 0.1050 - val_loss: 2.1250 - val_accuracy: 0.0907
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 2.1220 - accuracy: 0.1040
Epoch 00003: val_accuracy did not improve from 0.97407
3/3 [==============================] - 0s 9ms/step - loss: 2.0995 - accuracy: 0.1098 - val_loss: 2.0545 - val_accuracy: 0.1259
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 2.0577 - accuracy: 0.1400
Epoch 00004: val_accuracy did not improve from 0.97407
3/3 [========================

In [38]:
best_model = create_model()
best_model.load_weights('best_model.hdf5')
best_model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [39]:
results = best_model.evaluate(X_test, y_test)

17/17 [==============================] - 0s 2ms/step - loss: 0.1152 - accuracy: 0.9741


In [40]:
print('Accuracy: ', results[1])

Accuracy:  0.9740740656852722


## 2. Early stopping
- Cease training when model seems to overfit, i.e., target metric has stopped improving for certain epochs
- One can set **'patience'** parameter, which denotes number of epochs that model will endure without any improvements
    - e.g., if patience = 1, training will stop when metric has stopped improving for 2 epochs

In [41]:
callbacks = [EarlyStopping(monitor = 'accuracy', patience = 1)]

In [42]:
model = create_model()

In [43]:
# you could see that model stops training after few epochs
model.fit(X_train, y_train, epochs = 20, batch_size = 500, callbacks = callbacks, validation_data = (X_test, y_test))

Epoch 1/20
3/3 [==============================] - 0s 48ms/step - loss: 2.3684 - accuracy: 0.1050 - val_loss: 2.0260 - val_accuracy: 0.0815
Epoch 2/20
3/3 [==============================] - 0s 12ms/step - loss: 1.9589 - accuracy: 0.0979 - val_loss: 1.9083 - val_accuracy: 0.1093


## 3. Reduce learning rate
- In general, it is more desirable to lower down learning rate (learning rate decay) as training proceeds
- However, coming up with optimal learning rate decay scheme is not easy
- So, one of heuristics would be reducing learning rate when plateau is reached, in other words, when loss stops decreasing for certain number of epochs
    - learning rate will be decreased by factor of 'factor' parameter when objective metric has not improved for 'patience' parameter
<br>
<img src="https://i.ytimg.com/vi/s6jC7Wc9iMI/maxresdefault.jpg" style="width: 600px"/>

In [44]:
# halve learning rate when validation loss has not reduced for more than 5 epochs
callbacks = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 5)]

In [45]:
model = create_model()

In [46]:
model.fit(X_train, y_train, epochs = 20, batch_size = 500, callbacks = callbacks, validation_data = (X_test, y_test))

Epoch 1/20
3/3 [==============================] - 0s 48ms/step - loss: 2.9087 - accuracy: 0.0796 - val_loss: 2.2124 - val_accuracy: 0.0963
Epoch 2/20
3/3 [==============================] - 0s 10ms/step - loss: 2.0974 - accuracy: 0.1225 - val_loss: 1.9859 - val_accuracy: 0.2111
Epoch 3/20
3/3 [==============================] - 0s 9ms/step - loss: 1.9615 - accuracy: 0.2156 - val_loss: 1.9089 - val_accuracy: 0.2648
Epoch 4/20
3/3 [==============================] - 0s 10ms/step - loss: 1.8821 - accuracy: 0.3190 - val_loss: 1.7981 - val_accuracy: 0.3556
Epoch 5/20
3/3 [==============================] - 0s 9ms/step - loss: 1.7522 - accuracy: 0.3962 - val_loss: 1.6324 - val_accuracy: 0.4185
Epoch 6/20
3/3 [==============================] - 0s 11ms/step - loss: 1.5628 - accuracy: 0.4503 - val_loss: 1.4441 - val_accuracy: 0.5148
Epoch 7/20
3/3 [==============================] - 0s 9ms/step - loss: 1.3569 - accuracy: 0.5545 - val_loss: 1.2291 - val_accuracy: 0.6241
Epoch 8/20
3/3 [==============

In [47]:
results = model.evaluate(X_test, y_test)

17/17 [==============================] - 0s 1ms/step - loss: 0.1634 - accuracy: 0.9500


In [48]:
print('Accuracy: ', results[1])

Accuracy:  0.949999988079071
